# SDSS spectrum download

In [13]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
from time import sleep
import wget
import shutil
import os
import re
from bs4 import BeautifulSoup
import urllib.request

### references
# tutorial 1: https://medium.com/marketingdatascience/selenium%E6%95%99%E5%AD%B8-%E4%B8%80-%E5%A6%82%E4%BD%95%E4%BD%BF%E7%94%A8webdriver-send-keys-988816ce9bed
# tutorial 2: https://ithelp.ithome.com.tw/articles/10277622
# selenium Locating Elements: https://selenium-python.readthedocs.io/locating-elements.html
###

def ClearData():
    data = {
    'source_name': [],
    'class': [],
    'RA': [],
    'DEC': [],
    'z': [],
    'Type': [],
    'SDSS_spectrum_ID': [],
    'image_Donload_status': [],
    'spectrum_Donload_status': []
    }
    return data

### read in necessary parameters
table = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED code and data/data/BZCAT_Xmatch_ACC_list.csv')

table_name = table.name.values
table_class = table.Class.values
table_RA = table.RA.values
table_DEC = table.DEC.values
table_z = table.z.values

### find index of 
index = np.where(table_class == table_class[4])[0]

table_name = table_name[index]
table_class = table_class[index]
table_RA = table_RA[index]
table_DEC = table_DEC[index]
table_z = table_z[index]

df = ClearData()
    
for i in range(0, 1):  ### len(table_name)
               
    ### test paramenerts setting
    source_name = 'Test'
    source_z = 0.57
    source_RA = 20.423167
    source_DEC = 11.830719
    source_class = 'QSO RLoud flat radio sp.'
    
    ### for loop parameters setting
#     source_name = table_name[i]
#     source_z = table_z[i]
#     source_RA = table_RA[i] 
#     source_DEC = table_DEC[i] 
#     source_class = table_class[i]
    
    print('i =', i, ', source name:', source_name, ', RA =', source_RA, ', DEC =', source_DEC)
    
    #######################################################################################################
    ### start web crawler
    #######################################################################################################
    
    #######################################################################################################
    ### page 1  
    #######################################################################################################
    driver = webdriver.Chrome()

    ### location of data download
    driver.get("https://skyserver.sdss.org/dr18/VisualTools/navi") 
    
    RA_in = driver.find_element(By.ID, "ra")
    RA_in.clear()
    RA_in.send_keys(source_RA)
    
    DEC_in = driver.find_element(By.ID, "dec")
    DEC_in.clear()
    DEC_in.send_keys(source_DEC)
    
    Scale = driver.find_element(By.ID, "scale")
    Scale.clear()
    
    Update = driver.find_element(By.XPATH, "//button[@onclick = 'update();']")
    Update.click()
    
    object_type = driver.find_element(By.ID, "typeNear").text
    
    SDSS_ID = driver.find_element(By.ID, "objIdLabel").text
    print('SDSS ID:', SDSS_ID)
    
    df['source_name'].append( source_name )
    df['class'].append( source_class )
    df['RA'].append( source_RA )
    df['DEC'].append( source_DEC )
    df['z'].append( source_z )
    
    if not SDSS_ID == 'No objects within 0.2 arcmin':
        ### click "quick look"
        quick_look_link = 'https://skyserver.sdss.org/dr18/VisualTools/quickobj?objId=' + SDSS_ID
    
        driver.get(quick_look_link) 
        
        #######################################################################################################
        ### page 1  
        #######################################################################################################
        
        ### download spectrum image
        find_img_info = driver.find_elements(By.CLASS_NAME, "row")
        img_info = True

        for i in range(0, len(find_img_info)):
            text = find_img_info[i].text

            initial = 0
            content = []
            for k in range(0, len(text)): 
                if text[k:k+1] == '\n':
                    content.append(text[initial:k])
                    initial = k+1
                elif k == len(text)-1:
                    content.append(text[initial:k+1])

            for m in range(0, len(content)):
                if content[m] == 'The SDSS has not measured a spectrum for this object.':
                    img_info = False
            
        if img_info:
            img_link_text = driver.find_elements(By.TAG_NAME, "img") 

            image_link = img_link_text[2].get_attribute('src')

            wget.download(image_link)

            sleep(3)
            # shutil.move("/Users/87steven/Documents/ASIAA/Blazar SED code and data/SpecById.ashx", 
            #           "/Users/87steven/Documents/ASIAA/Blazar SED code and data/SDSS spectrum image/"+source_name+".png")
        
            ## download spectrum csv file
            driver.find_element(By.XPATH, "//span[@class = 'badge rounded-pill badge-warning']").click()

            sleep(10)
            # shutil.move("/Users/87steven/Downloads/format=csv.csv", 
            #             "/Users/87steven/Documents/ASIAA/Blazar SED code and data/SDSS spectrum/"+source_name+".csv")
            
            df['Type'].append( object_type )
            df['SDSS_spectrum_ID'].append( str(SDSS_ID)+'-' )
            df['image_Donload_status'].append( 'Complete' )
            df['spectrum_Donload_status'].append( 'Complete' )

            print('\n')
            print('=================================== File download succfully ===================================')
            
        else:
            df['Type'].append( 'N/A' )
            df['SDSS_spectrum_ID'].append( 'N/A' )
            df['image_Donload_status'].append( 'Not available' )
            df['spectrum_Donload_status'].append( 'Not available' )
            
            print('=================================== No file download ===================================')
    else:
        df['Type'].append( 'N/A' )
        df['SDSS_spectrum_ID'].append( 'N/A' )
        df['image_Donload_status'].append( 'Not available' )
        df['spectrum_Donload_status'].append( 'Not available' )

        print('=================================== No file download ===================================')
        
    driver.close()

    # CSVfile = f'/Users/87steven/Documents/ASIAA/Blazar SED code and data/SDSS spectrum/SDSS spectrum download log.csv'   
    #CSVfile = f'/Users/87steven/Downloads/test.csv'   
    dff = pd.DataFrame(df)      
    dff.to_csv(CSVfile, index = False)


i = 0 , source name: Test , RA = 20.423167 , DEC = 11.830719
SDSS ID: 1237678919139655754
100% [..............................................................................] 34599 / 34599

=================================== File download succfully ===================================


In [12]:
str(SDSS_ID)+'-' 

'1237678919139655754-'